# Extracting and Visualising Stock Data

## Importing necessary tools

In [3]:
!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install --upgrade plotly
!pip install html5lib
!pip install lxml
!install plotly

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
install: missing destination file operand after 'plotly'
Try 'install --help' for more information.


In [4]:
import yfinance as yf
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [5]:
import plotly.io as pio
pio.renderers.default = "iframe"

In [6]:
import warnings
warnings.filterwarnings

<function warnings.filterwarnings(action, message='', category=<class 'Warning'>, module='', lineno=0, append=False)>

### Question 1: Extracting Tesla stock data using yfinance

In [8]:
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-02-18 00:00:00-05:00,355.010010,359.100006,350.019989,354.109985,51631700,0.0,0.0
2025-02-19 00:00:00-05:00,354.000000,367.339996,353.670013,360.559998,67094400,0.0,0.0
2025-02-20 00:00:00-05:00,361.510010,362.299988,348.000000,354.399994,45965400,0.0,0.0
2025-02-21 00:00:00-05:00,353.440002,354.980011,334.420013,337.799988,73787800,0.0,0.0
2025-02-24 00:00:00-05:00,338.100006,342.397308,324.700012,329.619995,31788141,0.0,0.0


### Question 2: Extracting Tesla stock data using Webscraper

In [22]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text

# Parsing the data 

soup = BeautifulSoup(html_data, 'html.parser')

read_html_tesla_data = pd.read_html(str(soup))
tesla_revenue = read_html_tesla_data[1]
tesla_revenue.columns= ["Date", "Revenue"]
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace('$',"")
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',',"")
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

tesla_revenue.tail()

ProxyError: HTTPSConnectionPool(host='cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud', port=443): Max retries exceeded with url: /IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm (Caused by ProxyError('Unable to connect to proxy', OSError('Tunnel connection failed: 403 Forbidden')))

### Question 3: Extracting Gamestop's stock data using yfinance

In [ ]:
Gamestop = yf.Ticker("GME")
Gamestop_data = Gamestop.history(period="max")
Gamestop_data.head()
print("n\GameStop Stock Data:")
Gamestop_data.head()

### Question 4: Extracting Gamestop's stock data using Webscraping

In [ ]:
urlgme = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
gmedata = requests.get(urlgme).text
print(gmedata)

soupgme = BeautifulSoup(gmedata, 'html.parser')
price = soupgme.find('fin-steamer', {'data-symbol': 'GME', 'data-field': 'regularMarketPrice'}).text
stock_data = {
    'Price': price,
    'Previous_Close': soupgme.find('td', {'data-test': 'PREV_CLOSE-value'}).text,
'Open': soupgme.find('td', {'data-test': 'OPEN-value'}).text,
'Volume': soupgme.find('td', {'data-test': 'TD_VOLUME-value'}).text
}

dfgme = pd.DataFrame([gme_data])
print("/nExtracted Data:")
print(dfgme)

read_html_gme_data = pd.read_html(str(soup1))
gme_revenue = read_html_gme_data[1]
gme_revenue.columns= ["Date", "Revenue"]
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace('$',"")
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',',"")

## Defining Graph function

In [ ]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()
    from IPython.display import display, HTML
    fig_html = fig.to_html()
    display(HTML(fig_html))

### Question 5: Plot Tesla Stock Graph

In [ ]:
make_graph(tesla_data,tesla_revenue , 'TESLA')

### Question 6: Plot GME Stock Graph

In [ ]:
make_graph(gme_data, gme_revenue, 'GameStop')